In [17]:
from pathlib import Path
import pandas as pd
import pickle
import gzip

In [3]:
sif_path = Path('/data/MLDSST/nielsond/cogmood/analysis/SUPREME/Analysis/Flanker/flanker.sif')
swarm_dir = Path('/data/MLDSST/nielsond/cogmood/swarm/')
swarm_cmd_dir = swarm_dir / 'cmds'
swarm_log_dir = swarm_dir / 'log'
clean_csv_dir = Path('/data/MLDSST/nielsond/cogmood/data/20250611_pilot/to_model')
dq = pd.read_csv(clean_csv_dir / 'data_quality.csv')
res_dir = Path('/data/MLDSST/nielsond/cogmood/data/20250611_pilot/model_res_20260127')
res_dir.mkdir(exist_ok=True)

In [6]:
good_flkr_csvs = [clean_csv_dir / f'flkr-{sid}.csv' for sid in dq.loc[dq.good10_flkr, 'sub_id'].values]

In [5]:
cmds = []
for sub_file in good_flkr_csvs:
    cmd = f"""rsync -avch /data/MLDSST/nielsond/cogmood/analysis/SUPREME/Analysis/Flanker /lscratch/$SLURM_JOBID/; \
    rsync -avch {sub_file} /lscratch/$SLURM_JOBID/Flanker/; \
    cd /lscratch/$SLURM_JOBID/Flanker; \
    mkdir -p /lscratch/$SLURM_JOBID/cache/pycuda; \
    ls -ltah /lscratch/$SLURM_JOBID/Flanker/flanker.sif; \
    export MPLBACKEND=pdf; \
    apptainer run --nv -B /lscratch/$SLURM_JOBID/Flanker:/Flanker \
        -B /lscratch/$SLURM_JOBID/cache:/data/nielsond/.cache \
        /lscratch/$SLURM_JOBID/Flanker/flanker.sif {sub_file.parts[-1]} ; \
    rsync -avch flanker_{sub_file.stem}.tgz {res_dir}/; 
    """
    cmds.append(cmd)

In [6]:
cmd_file = swarm_cmd_dir/'pilot_flkr6'
cmd_file.write_text('\n'.join(cmds))

19149

In [8]:

! swarm --time=12:00:00 -g 30 -t 20 -m apptainer --gres=gpu:1,lscratch:5 --partition=gpu --job-name=flkr_pilot --logdir={swarm_log_dir} --sbatch="--constraint=gpuk80\|gpuv100" {cmd_file}

10308342


In [18]:
def load_results(filename):
    """
    Load in a simulation that was saved to a pickle.gz.
    """
    gf = gzip.open(filename, 'rb')
    res = pickle.loads(gf.read(), encoding='latin1')
    gf.close()
    return res


In [26]:
cmds = []
for sub_file in good_flkr_csvs:
    result_good=False
    outfile = res_dir / f'flanker_{sub_file.stem}.tgz'
    if outfile.exists():
        nsims = load_results(outfile)['particles'].shape[0]
        if nsims == 2001:
            result_good = True
    if not result_good:
        cmd = f"""rsync -avch /data/MLDSST/nielsond/cogmood/analysis/SUPREME/Analysis/Flanker /lscratch/$SLURM_JOBID/; \
        rsync -avch {sub_file} /lscratch/$SLURM_JOBID/Flanker/; \
        cd /lscratch/$SLURM_JOBID/Flanker; \
        mkdir -p /lscratch/$SLURM_JOBID/cache/pycuda; \
        ls -ltah /lscratch/$SLURM_JOBID/Flanker/flanker.sif; \
        export MPLBACKEND=pdf; \
        apptainer run --nv -B /lscratch/$SLURM_JOBID/Flanker:/Flanker \
            -B /lscratch/$SLURM_JOBID/cache:/data/nielsond/.cache \
            /lscratch/$SLURM_JOBID/Flanker/flanker.sif {sub_file.parts[-1]} ; \
        rsync -avch flanker_{sub_file.stem}.tgz {res_dir}/; 
        """
        cmds.append(cmd)

In [24]:
cmd_file = swarm_cmd_dir/'pilot_flkr6_fails'
cmd_file.write_text('\n'.join(cmds))

7253

In [25]:
! swarm --time=12:00:00 -g 30 -t 20 -m apptainer --gres=gpu:1,lscratch:5 --partition=gpu --job-name=flkr_pilot --logdir={swarm_log_dir} --sbatch="--constraint=gpuk80\|gpuv100" {cmd_file}

10387043


Just some commands that might be needed for debuging:

```
apptainer shell --nv -B /lscratch/$SLURM_JOBID/Flanker:/Flanker -B /home/nielsond:/home/nielsond -B /lscratch/$SLURM_JOBID/cache:/data/nielsond/.cache ./flanker.sif 

sinteractive --gres=gpu:k80:1,lscratch:20 --mem=40g -c20 --time=24:00:00
```